<a href="https://colab.research.google.com/github/AdamsD02/Music-Genre-Classification/blob/main/mgc_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone "https://github.com/AdamsD02/Music-Genre-Classification.git"


Cloning into 'Music-Genre-Classification'...
remote: Enumerating objects: 1027, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1027 (delta 8), reused 0 (delta 0), pack-reused 1012 (from 2)
Receiving objects: 100% (1027/1027), 1.13 GiB | 32.03 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (1003/1003), done.


In [17]:
%cd "Music-Genre-Classification"

/content/Music-Genre-Classification


In [18]:
%ls

config.py  features_30_sec.csv  genres/  mgc_code.ipynb  README.md


 Install Librosa library for audio analysis and manipulation

In [ ]:
pip install librosa

In [30]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [27]:
!git restore --staged config.py

In [29]:
!git config --global user.email "jdsouzaadams@gmail.com"
!git config --global user.name "AdamsD02"
!git add .gitignore
!git commit -m "Add config.py to .gitignore"

[main 7af36ee] Add config.py to .gitignore
 1 file changed, 2 insertions(+)
 create mode 100644 .gitignore


In [31]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address


### Import Libraries

In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


## Read .csv file containing data for Classifying Genres

In [2]:
df = pd.read_csv("/content/Music-Genre-Classification/features_30_sec.csv")
# df.head()
# list dataset features
print(df.columns.tolist())

['filename', 'length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var', 'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean', 'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var', 'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo', 'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean', 'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var', 'label']


### Manipulate dataframe to:
*   Separate target(X) and features(y)
*   Split into traning and testing (80-20)

In [3]:
X = df.iloc[:, 2:59] # chroma_stft_mean to mfcc20_var

# Encode non-numerical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['label'])

print(X.shape)
print(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print("X_train size: ", len(X_train))
print("X_test size: ", len(X_test))

(1000, 57)
[0 1 2 3 4 5 6 7 8 9]
X_train size:  800
X_test size:  200


### Use LightGBM as Model Classification and training

In [ ]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
# clf = lgb.LGBMClassifier(force_col_wise=False)
clf.fit(X_train, y_train)


### Testing and Predictions

In [5]:
# predict the results
y_pred = clf.predict(X_test)

predicted_genres = le.inverse_transform(y_pred)

mse = np.mean((y_pred - y_test) ** 2)
count = 0
print("Mse: ", mse)
# print(y_pred)
for i in range(len(y_pred)):
  if y_pred[i] - y_test[i] == 0:
    count += 1
  # print("Predicted: ", predicted_genres[i], "\t| Actual: ", le.inverse_transform([y_test[i]]))

print("Accuracy: ", count/len(y_pred))


Mse:  7.76
Accuracy:  0.73


In [6]:
from google.colab import files

myfile = files.upload()

Saving Ai Se Eu Te Pego.mp3 to Ai Se Eu Te Pego.mp3


## Feature Extraction from Uploaded Audio

1.   Load and get sampling rate (sr)
2.   trim audio (y)
3.   sr should match fixed value in dataset, if not then resample
4.   find chroma_stft, then its mean & variance
5.   find rms, then its mean & var
6.   find spectral_centroid, then its mean and var
7.   find spectral_bandwidth, then its mean & var
8.   roll_off, then its mean & var
9.   zero-crossing-rate
10.  harmonics, percetrual
11.  tempo
12.  mfcc with n_mfcc=20


In [7]:

filename = list(myfile.keys())[0]
print("File uploaded:", filename)

# If needed, get full path
filepath = os.path.join(os.getcwd(), filename)
print("Full path:", filepath)

# Load an MP3 file
y, sr = librosa.load(filepath, sr=None)  # sr=None keeps original sample rate
print("Sampling rate:", sr)

if sr != 22050:
  y = librosa.resample(y=y, orig_sr=sr, target_sr=22050)
  sr = 22050
  print("New Sampling Rate: ", sr)

# n_fft=2048, hop_length=512

features = []

#get chroma_mean and chroma_var
chroma = librosa.feature.chroma_stft(y=y, sr=sr)
chroma_mean = float(np.mean(chroma))
chroma_var = float(np.var(chroma))
print("Chroma Mean:", chroma_mean, "\t | Chroma Variance:", chroma_var)
features.append(chroma_mean)
features.append(chroma_var)

#get rms_mean & rms_var
rms = librosa.feature.rms(y=y)
rms_mean = float(np.mean(rms))
rms_var = float(np.var(rms))
print("RMS Mean:", rms_mean, "\t | RMS Variance:", rms_var)
features.append(rms_mean)
features.append(rms_var)

#get spectral-centroid mean & var
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_cent_mean = float(np.mean(spec_cent))
spec_cent_var = float(np.var(spec_cent))
print("Spectral Centroid Mean:", spec_cent_mean, "\t | Spectral Centroid Variance:", spec_cent_var)
features.append(spec_cent_mean)
features.append(spec_cent_var)

#get spectral-bandwidth mean & var
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
spec_bw_mean = float(np.mean(spec_bw))
spec_bw_var = float(np.var(spec_bw))
print("Spectral Bandwidth Mean:", spec_bw_mean, "\t | Spectral Bandwidth Variance:", spec_bw_var)
features.append(spec_bw_mean)
features.append(spec_bw_var)

#get roll-off mean & var
spec_roll_off = librosa.feature.spectral_rolloff(y=y, sr=sr)
spec_roll_off_mean = float(np.mean(spec_roll_off))
spec_roll_off_var = float(np.var(spec_roll_off))
print("Roll-off Mean:", spec_roll_off_mean, "\t | Roll-off Variance:", spec_roll_off_var)
features.append(spec_roll_off_mean)
features.append(spec_roll_off_var)

#get zero-crossing-rate mean & var
zcr = librosa.feature.zero_crossing_rate(y)
zcr_mean = float(np.mean(zcr))
zcr_var = float(np.var(zcr))
print("Zero Crossing Rate Mean:", zcr_mean, "\t | Zero Crossing Rate Variance:", zcr_var)
features.append(zcr_mean)
features.append(zcr_var)

#get harmonics & percetural mean & var
harm, perc = librosa.effects.hpss(y)
harm_mean = float(np.mean(harm))
harm_var = float(np.var(harm))
perc_mean = float(np.mean(perc))
perc_var = float(np.var(perc))
print("Harmonics Mean:", harm_mean, "\t | Harmonics Variance:", harm_var)
print("Percetural Mean:", perc_mean, "\t | Percetural Variance:", perc_var)
features.append(harm_mean)
features.append(harm_var)
features.append(perc_mean)
features.append(perc_var)

#get tempo
tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
print("Tempo:", tempo)
features.append(float(tempo[0]))

#get mfcc_mean & mfcc_var 1 to 20
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
print("MFCC: ", mfcc.shape)
for i in range(20):
  features.append(float(np.mean(mfcc[i])))
  features.append(float(np.var(mfcc[i])))
  # print("MFCC" + str(i+1) + " Mean:", np.mean(mfcc[i]), "\t | MFCC" + str(i+1) + " Variance:", np.var(mfcc[i]))



print("Audio shape:", y.shape)
print("Sample rate:", sr)
print("\n Features:\n", features)

File uploaded: Ai Se Eu Te Pego.mp3
Full path: /content/Ai Se Eu Te Pego.mp3
Sampling rate: 44100
New Sampling Rate:  22050
Chroma Mean: 0.3597618043422699 	 | Chroma Variance: 0.08780751377344131
RMS Mean: 0.18586300313472748 	 | RMS Variance: 0.003760875202715397
Spectral Centroid Mean: 2204.03600192656 	 | Spectral Centroid Variance: 595197.0408157627
Spectral Bandwidth Mean: 2424.075389395411 	 | Spectral Bandwidth Variance: 182703.56401606588
Roll-off Mean: 4628.1986880751265 	 | Roll-off Variance: 2328196.6249690917
Zero Crossing Rate Mean: 0.09927859497927789 	 | Zero Crossing Rate Variance: 0.004396040521885233
Harmonics Mean: -6.0795602621510625e-05 	 | Harmonics Variance: 0.01746355928480625
Percetural Mean: -0.00044534343760460615 	 | Percetural Variance: 0.009708119556307793
Tempo: [95.703125]
MFCC:  (20, 7118)
Audio shape: (3643904,)
Sample rate: 22050

 Features:
 [0.3597618043422699, 0.08780751377344131, 0.18586300313472748, 0.003760875202715397, 2204.03600192656, 595197

In [8]:
y_pred = clf.predict([features])

predicted_genres = le.inverse_transform(y_pred)
print("Predicted Genre: ", predicted_genres)

Predicted Genre:  ['rock']


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


## GUI

In [11]:
!pip install flask pyngrok

In [14]:
from flask import Flask
from pyngrok import ngrok, conf
from config import NGROK_TOKEN

#setup ngrok authotoken
token = NGROK_TOKEN
ngrok.set_auth_token(token)

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "Hello, World!"
public_url = ngrok.connect(5000)
print(" * ngrok tunnel URL:", public_url)
app.run(port=5000)

ModuleNotFoundError: No module named 'config'